In [ ]:
import kwant
import semicon

import numpy as np
import scipy.linalg as la
import scipy.sparse
import scipy.sparse.linalg as sla

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

import codes.higher_order_lowdin as lowdin

In [ ]:
# %load_ext line_profiler

# k.p example of using the Lowdin perturbation

This example requires [semicon](https://gitlab.kwant-project.org/semicon/semicon) to be installed.

It should be as easy as 
```
pip install git+https://gitlab.kwant-project.org/semicon/semicon.git
```

In [ ]:
try:
    import semicon
except ImportError:
    print("Semicon should be installed to run this notebook.")

In [ ]:
import kwant
import scipy
import sympy
import semicon


print("semicon version:", semicon.__version__)
print("kwant version:", kwant.__version__)
print("scipy version:", scipy.__version__)
print("sympy version", sympy.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=kwant.KwantDeprecationWarning)

# Prepare k.p model and solve it exactly

In [ ]:
gamma_0 = 1.0

# Get the k.p model for ZincBlende material
model = semicon.models.ZincBlende(
    components=('foreman',),
    default_databank='lawaetz'
)
kpham = model.hamiltonian

# Get parameters for InAs
InAs = model.parameters('InAs').renormalize(new_gamma_0=gamma_0)

display(kpham)



In [ ]:
# Discretize the model to get a kwant tight-binding model
grid_spacing = 0.5
R = 10

tbham = kwant.continuum.discretize(kpham, grid=grid_spacing)

# Fill an infinite wire with circular cross section with the model
shape = lambda site: la.norm(site.pos) < R
tz = np.array(tbham.symmetry.periods)[-1]
wire = kwant.Builder(symmetry=kwant.TranslationalSymmetry(tz))
wire.fill(tbham, shape, start=(0, 0, 0))
kwant.plotter.plot(wire, num_lead_cells=10, pos_transform=lambda pos: pos[:2]);

# Get the unit cell and hopping matrices
wiref = wire.finalized()
Hos = wiref.cell_hamiltonian(params=InAs, sparse=True).tocsr()
Hos.eliminate_zeros()
Hhop = wiref.inter_cell_hopping(params=InAs, sparse=True).tocsr()
Hhop.eliminate_zeros()

# Make second order model in k_z
H0 = lowdin.Model({1: Hos})
Hkz = lowdin.Model({'k_z': (1j * Hhop - 1j * Hhop.T.conj()), 'k_z**2': -(Hhop + Hhop.T.conj())})
H = H0 + Hkz

# exact eigenenergies
es = []
ks = np.linspace(-0.5, 0.5, 21)
for k in ks:
    e = scipy.sparse.linalg.eigsh(H.subs({'k_z': k})[1], return_eigenvectors=False, sigma=0.5, k=6)
    es.append(e)

es = np.array(es)
    
plt.plot(ks, es, '-');

In [ ]:
# Add electric field perturbations

xop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[0] * np.eye(8), sum=True).tocoo().tocsr()
xop.eliminate_zeros()
yop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[1] * np.eye(8), sum=True).tocoo().tocsr()
yop.eliminate_zeros()

V = lowdin.Model({'E_x': -xop, 'E_x': -yop})

In [ ]:
# Make perturbation basis of two lowest states
E0, vecsA = scipy.sparse.linalg.eigsh(Hos, return_eigenvectors=True, sigma=0.5, k=2)
vecsA, _ = la.qr(vecsA, mode='economic')

model0 = lowdin.effective_model(H0, Hkz + V, evec_A=vecsA, order=2)

In [ ]:
model0.around().tosympy()

In [ ]:
# remove some keys
intersting_keys = lowdin._interesting_keys((Hkz + V).keys(), order=2)
intersting_keys -= set(sympy.sympify(['k_z**3', 'k_z**4', 'E_x**2', 'E_y**2']))
model0 = lowdin.effective_model(H0, Hkz + V, evec_A=vecsA, order=2, interesting_keys=intersting_keys)
model0.around().tosympy()

In [ ]:
# Make perturbation basis of second lowest states
E0, vecsA = scipy.sparse.linalg.eigsh(Hos, return_eigenvectors=True, sigma=0.8, k=4)
vecsA, _ = la.qr(vecsA, mode='economic')

model1 = lowdin.effective_model(H0, Hkz + V, evec_A=vecsA, order=2, interesting_keys=intersting_keys)

In [ ]:
model1.around().tosympy()

In [ ]:
# effective eigenenergies
es1 = []
es0 = []
ks2 = np.linspace(-0.5, 0.5, 101)
for k in ks2:
    e = la.eigh((model0).subs({'k_z': k, 'E_x': 0})[1], eigvals_only=True)
    es0.append(e)
    e = la.eigh((model1).subs({'k_z': k, 'E_x': 0})[1], eigvals_only=True)
    es1.append(e)
es0 = np.array(es0)
es1 = np.array(es1)
    
plt.plot(ks, es, '--')
plt.plot(ks2, es0, '-')
plt.plot(ks2, es1, '-')

In [ ]:
# Use 12 states exactly
# Make perturbation basis of two lowest states
E0, vecs = scipy.sparse.linalg.eigsh(Hos, return_eigenvectors=True, sigma=0.5, k=12)
vecs, _ = la.qr(vecs, mode='economic')

model0 = lowdin.effective_model(H0, Hkz + V, evec_A=vecs[:, :2], order=2, interesting_keys=intersting_keys)
display(model0.around().tosympy())

model1 = lowdin.effective_model(H0, Hkz + V, evec_A=vecs[:, 2: 6], order=2, interesting_keys=intersting_keys)
display(model1.around().tosympy())

# effective eigenenergies
es1 = []
es0 = []
ks2 = np.linspace(-0.5, 0.5, 101)
for k in ks2:
    e = la.eigh((model0).subs({'k_z': k, 'E_x': 0})[1], eigvals_only=True)
    es0.append(e)
    e = la.eigh((model1).subs({'k_z': k, 'E_x': 0})[1], eigvals_only=True)
    es1.append(e)
es0 = np.array(es0)
es1 = np.array(es1)
    
plt.plot(ks, es, '--')
plt.plot(ks2, es0, '-')
plt.plot(ks2, es1, '-')

### Magnetic field perturbation

In [ ]:
from codes.combine import apply_peierls_to_template

In [ ]:
# Discretize the model to get a kwant tight-binding model
grid_spacing = 0.5
R = 2

tbham = kwant.continuum.discretize(kpham, grid=grid_spacing)

# Apply Peierels substitution to have magnetic field
tbham2 = apply_peierls_to_template(tbham)

# Fill an infinite wire with circular cross section with the model
shape = lambda site: la.norm(site.pos) < R
tz = np.array(tbham2.symmetry.periods)[-1]
wire = kwant.Builder(symmetry=kwant.TranslationalSymmetry(tz))
wire.fill(tbham2, shape, start=(0, 0, 0))
kwant.plotter.plot(wire, num_lead_cells=10, pos_transform=lambda pos: pos[:2]);

params = {**InAs, 'e':1, 'B_x': 0, 'B_y': 0, 'B_z': 0}

# Get the unit cell and hopping matrices
wiref = wire.finalized()
kwant.plotter.bands(wiref, params=params)
Hos = wiref.cell_hamiltonian(params=params, sparse=True).tocsr()
Hhop = wiref.inter_cell_hopping(params=params, sparse=True).tocsr()

In [ ]:
# Generate magnetic field perturbation terms
deltaB = 1
HBos = 0
HBhop = 0
for B in ['B_x', 'B_y', 'B_z']:
    paramsB = params.copy()
    paramsB[B] = deltaB
    valB = (wiref.cell_hamiltonian(params=paramsB, sparse=True).tocsr()
            - wiref.cell_hamiltonian(params=params, sparse=True).tocsr()) / deltaB
    HBos += lowdin.Model({B: valB})
    valB = (wiref.inter_cell_hopping(params=paramsB, sparse=True).tocsr()
            - wiref.inter_cell_hopping(params=params, sparse=True).tocsr()) / deltaB
    HBhop += lowdin.Model({B: valB})